# Purpose of this notebook

This notebook serves to demonstrate the following workflow to help introduce new users to the UNSAFE framework, and to allow them to test that the code does what it says it does. 

1) Configure the working directory structure and workflow parameters;
2) If you want to make use of the downloading functionality in UNSAFE, download and unzip data;
3) Subset the full structure inventory to single-family structures and convert the data to a GeoDataFrame;
4) Specify a spatial extent of your study area (i.e. county shapefile or a study boundary) and process reference files through clipping (e.g. census tract data);
5) Process expert DDFs for use in ensembles;
6) Process social vulnerability data by linking it with corresponding reference data (e.g. linking Climate and Economic Justice Screening Tool with census tracts);
7) Prepare the National Flood Hazard Layer data for identifying structure location in and outside of the federal floodplain;
8) Link structures to all vector spatial data;
9) Link structures to inundation data, provided as raster(s);
10) Prepare the structure inventory for loss estimation (this base inventory can be used for estimating losses without uncertainty);
11) Generate an ensemble of plausible structure realizations based on several parameters users can specify;
12) Estimate expected annual losses for each ensemble member for a set of design events. Each ensemble member has a unique draw from the DDF distribution. Users can also estimate expected annual losses without uncertainty in exposure and vulnerability. 


We also provide code for producing some visualizations. 

# Configure

In [ ]:
# These are options we like setting for working with jupyter notebooks. Totally optional
%load_ext autoreload
%autoreload 2

We load packages (mostly from UNSAFE) to help us get through the workflow.

In [ ]:
# Packages
import os
from os.path import join
from pathlib import Path
import yaml
from yaml.loader import SafeLoader
import pandas as pd

from unsafe.download import *
from unsafe.files import *
from unsafe.const import *
from unsafe.unzip import *
from unsafe.exp import *
from unsafe.ddfs import *
from unsafe.ensemble import *

Next, we configure spatial units like county fips. Right now, the county is the unit of analysis for UNSAFE. The county fips code is used to download data from the national structure inventory API, and for organzing the directory structure. This facilitates distributed county-level analyses (though we have not tested this yet, and there are likely some things to work out before then). We also keep track of the state fips code (the first two digits of the county code) for downloading data and organizing the file directory. The state abbrevation for a county can be helpful (though it's not currently used). We also have the "Nation" indicator, since damage functions have been produced in other countries. Keeping track of nation is probably not a feature that will remain in UNSAFE in future versions. In future versions, we plan to have state fips and state abbrevation inferred from the user-supplied FIPS code. 

In [ ]:
# Name the fips, statefips, stateabbr, and nation that
# we are using for this analysis
# We pass these in as a list even though the framework currently
# processes a single county so that it can facilitate that
# expansion in the future
# TODO - could make sense to define these in the future
# in json or other formats instead of as input in code
fips_args = {
    'FIPS': ['42101'], 
    'STATEFIPS': ['42'],
    'STATEABBR': ['PA'],
    'NATION': ['US']
}
FIPS = fips_args['FIPS'][0]
NATION = fips_args['NATION'][0]


Most of the configuration for UNSAFE is about setting up the file directory structure. Users identify the absolute directory to the root of their project directory and store this in ABS_DIR. This is where we recommend storing a config.yaml file in a `congig/` directory. Below, we show all the file directory configurations. 

In [ ]:
# We need to pass in a config file that sets up
# constants and the structure for downlading data
# For the directory structure of our case study, 
# we use the following, but you may
# have to specify a different Path
ABS_DIR = Path().absolute().parents[0]


# We are updating the config filename because for this example
# we are using different hazard data. The HAZ_FILEN parameter
# will change values. Everything else is the same. 
CONFIG_FILEP = join(ABS_DIR, 'config', 'config_partial.yaml')
# Open the config file and load
with open(CONFIG_FILEP) as f:
    CONFIG = yaml.load(f, Loader=SafeLoader)

# We can also specify the filepath to the
# raw data directory
FR = join(ABS_DIR, "data", "raw")

# And external - where our hazard data should be
FE = join(FR, "external")

# Set up interim and results directories as well
# We already use "FR" for raw, we use "FO" 
# because you can also think of results
# as output
FI = join(ABS_DIR, "data", "interim")
FO = join(ABS_DIR, "data", "results")

# "Raw" data directories for exposure, vulnerability (vuln) and
# administrative reference files
EXP_DIR_R = join(FR, "exp")
VULN_DIR_R = join(FR, "vuln")
REF_DIR_R = join(FR, "ref")
# Haz is for depth grids
HAZ_DIR_R = join(FE, "haz")
# Pol is for NFHL
POL_DIR_R = join(FR, "pol")

# Unzip directory 
UNZIP_DIR = join(FR, "unzipped")

# We want to process unzipped data and move it
# to the interim directory where we keep
# processed data
# Get the filepaths for unzipped data
# We unzipped the depth grids (haz) and 
# ddfs (vuln) into the "external"/ subdirectory
HAZ_DIR_UZ = join(UNZIP_DIR, "external", "haz")
POL_DIR_UZ = join(UNZIP_DIR, "pol")
REF_DIR_UZ = join(UNZIP_DIR, "ref")
VULN_DIR_UZ = join(UNZIP_DIR, "external", "vuln")

# "Interim" data directories
EXP_DIR_I = join(FI, "exp")
VULN_DIR_I = join(FI, "vuln")
REF_DIR_I = join(FI, "ref")
# Haz is for depth grids
HAZ_DIR_I = join(FI, "haz")
# Pol is for NFHL
POL_DIR_I = join(FI, "pol")

Next, we provide detailed comments on the other things in the config.yaml file.

In [ ]:
# Get the files we need downloaded
# These are specified in the "download" key 
# in the config file
# We transpose because one of the utils
# needs to return a list of the output files
DOWNLOAD = pd.json_normalize(CONFIG['download'], sep='_').T

# Wildcards for urls. For example, {FIPS} is an element in this
# list because in the urls supplied in the download dictionary, 
# this is what we are going to replace with the FIPS code
# for our analysis. We don't assume that the wildcard is the
# only text in between brackets in a url (it happens to be the
# case for this case study) so we think it's useful to have
# this list pre-configured. 
URL_WILDCARDS = CONFIG['url_wildcards']

# Get the file extensions for api endpoints
# In our case study, this is only for downloading from the NSI
API_EXT = CONFIG['api_ext']

# The data from the NSI is in .json format
# and we couldn't find the coordinate reference system
# through GET requests (we may have missed how). However,
# we were able to find metadata online that indicates 
# the CRS. 
NSI_CRS = CONFIG['nsi_crs']

# Dictionary of ref_names
# When we download tracts, block groups, etc. 
# from the TIGER endpoints, they can sometimes have a lot of
# characters. We find it helpful to standardize the
# names (i.e. block instead of tabblock20), but this
# can be customized to your preferences. 
REF_NAMES_DICT = CONFIG['ref_names']

# Dictionary of ref_id_names
# We will run the same processes on all the reference data
# like reprojecting and clipping to our study boundaries.
# We also will merge tabular data that is designed at
# certain administrative boundaries with attribute
# joins. This dictionary converts the names like GEOID
# to tract_id. 
REF_ID_NAMES_DICT = CONFIG['ref_id_names']

# Coefficient of variation
# for structure values
# This is what we scale the structure value by
# to get the standard deviation we draw from
COEF_VARIATION = CONFIG['coef_var']

# First floor elevation dictionary
# This maps foundation types to the triangular distributions
# for first-floor elevation
FFE_DICT = CONFIG['ffe_dict']

# Number of states of the world
# This is the number of ensemble members
N_SOW = CONFIG['sows']

# The hazard configuration will vary on a case study by
# case study basis. There may be different configuration
# parameters that you need to specify depending on how
# the hazard data you're using is structured. We hope
# to provide techniques to systematically accommodate 
# certain inundation model outputs as we gain more experience 
# coupling UNSAFE with different models
# The below structure can work on any depth grids from the FEMA
# Flood Risk Database. We specify this case study
# to focus on the riverine flooding products and for the
# 500, 100, 50, and 10 year return periods. 

# Get hazard model variables
# Get Return Period list
RET_PERS = CONFIG['RPs']
HAZ_FILEN = CONFIG['haz_filename']
# Get CRS for depth grids
HAZ_CRS = CONFIG['haz_crs']

# Download (and unzip) data

We added download functionality to UNSAFE because it can  help make an analysis more reproducible. Instead of manually downloading hazard, exposure, and vulnerability data from different sources, much of (and sometimes all of) the data needed for a standard risk assessment can be specified using URL or API endpoints. 

We download data with `download_raw(files, wcard_dict, fr, api_ext)` and then unzip data -- both downloaded and user uploaded -- using `unzip_raw(fr, unzip_dir)`. In this partial data example, some of the data is manually uploaded in the `philadelphia_frd/data/raw/external/` directory. When you download UNSAFE, it comes with `external/haz/dg_clip.zip` and `external/vuln/ddfs.zip`. The first directory corresponds to the hazard data we will use for the case study, which originally comes from the Federal Emergency Managemeng Agency (FEMA) Philadelphia Flood Risk Database. You can manually download the data we used by going to [the FEMA Flood Map Service Center: Search All Products](https://msc.fema.gov/portal/advanceSearch#searchresultsanchor) and searching for "FRD_02040202_PA_GeoTIFFs" in the Product ID search box. This data is over 2 GB. You can use this data to follow the `full_data_example.ipynb` notebook. We clipped this raw data to a small spatial extent to allow for the partial example shown here. The second directory, `external/vuln/ddfs.zip`, corresponds to the Depth-Damage Functions, downloaded from [here](https://zenodo.org/records/10027236). This Zenodo repository is well-documented and includes a detailed explanation of where the records are originally from. We manually download these and store them in the data directory for the case study because the URL endpoints for most FEMA Flood Map Service Center products are difficult to identify, and the process for accessing the Zenodo API is somewhat complex. 

The `files` object (specified in CONFIG['DOWNLOAD']) should be structured in a nested dictionary in the following format: SPATIAL_UNIT -> ENDPOINT_TYPE -> COMPONENT -> FILENAME: ENDPOINT. As an example, to download the structures for a county from the NSI, you would specify `FIPS: api: exp: nsi: "https://nsi.sec.usace.army.mil/nsiapi/structures?fips={FIPS}"`. The `wcard_dict` is specified in our configuration step above, and will convert {FIPS} to our county code. "api" tells the function that we are downloading data from an api endpoint (we could also specify url), and exp tells the function that this is an exposure dataset and should be organized in the working directory accordingly. Other "COMPONENT"s include vuln for vulnerability, haz for hazard, pol for policy (e.g. National Flood Hazard Layer), and ref for reference (e.g. censust tract boundaries). We clarify when vulnerability refers to social or physical vulnerability with an additional nested dictionary, like COMPONENT -> SUBCOMPONENT -> FILENAME: ENDPOINT. 

Some endpoints need to be manually specified (e.g. the URL for the National Flood Hazard Layer) because we do not know how to uniquely identify the corresponding data (and the FEMA Help Desk did not respond to our emails asking for guidance). We believe it is a more reproducible practice to specify the download links when possible, as opposed to manually downloading files and uploading them to the working directory. However, users do not have to use the download functionality in UNSAFE. Users could treat all data as external to the UNSAFE workflow, and would configure the working environment accordingly. We recommend putting this data in `ABS_DIR/data/raw/external/` with subdirectories for a specific COMPONENT, but you can modify the filepaths defined above if you'd like to put this data somewhere else. 

`fr` corresponds to the relative filepath to the raw data directory. UNSAFE currently gives users the options to customize the directory structure in the configuration step, but enforces the nested directory structure within that which separates data by SPATIAL_UNIT and COMPONENT. We find that the raw, interim, and results structure provided in this tutorial is helpful for many types of risk assessments. 

In [ ]:
# The unsafe.download library provides us with
# convenient functions for quickly downloading data from the sources
# we specified in the config.yaml

# URL_WILDCARDS has entries like {FIPS} which we want to replace
# with the county code that is in a URL for downloading. 
# We create a dictionary of these mappings from our fips_args
# dictionary. This is what we need to use the download_raw()
# function 

wcard_dict = {x: fips_args[x[1:-1]][0] for x in URL_WILDCARDS}
download_raw(DOWNLOAD, wcard_dict,
             FR, API_EXT)

In [ ]:
# We call unzip_raw from util.unzip
# This will unzip files we downloaded, but also 
# .zip files that we uploaded to raw/external/
# or wherever you put these files. We use glob.glob
# to search for .zip directories in raw/, so as
# long as you put external data somewhere in raw/
# this should work. That said, we have only
# tested this function when the directory
# is structured as we do in this example.
unzip_raw(FR, UNZIP_DIR)

Both of the functions we just called do a lot of work under the hood for us, calling helper functions in `unsafe.download` and `unsafe.unzip`

# Prepare data for ensemble

This section comprises steps 3 through 9 mentioned at the top of this notebook.

## Exposure

Now that we have downloaded the NSI records for the county, we are going to do some preprocessing for generating our ensemble.

First, we will call `get_nsi_geo()` to use the spatial coordinates from the NSI and prepare a GeoDataFrame. We need the data in this format for linking other attributes needed for loss estimation with the NSI, like flood depths. 

Next, we subset the records from the NSI using SQL-like querying on our pandas DataFrame. 

These steps give us a GeoDataFrame of single family residences for our ensemble. 

In [ ]:
# For this case study, we want single family houses from the
# national structure inventory. We will call functions from exp.py
# that takes the raw nsi data and converts it to a gdf
# Then we will grab our properties of interest using the RES1
# code for the 'occtype' variable. In addition, this case study
# will look at properties <= 2 stories because these are
# the properties we can represent structural uncertainty in
# depth-damage relationships for

# This function asks us to pass in the FIPS code,
# the coordinate reference system for the structure inventory
# and the subdirectory where we put the raw structure inventory
# data. We defined all of these arguments in our configuration
# steps above. 
nsi_gdf = get_nsi_geo(FIPS, NSI_CRS, EXP_DIR_R)

# Set the values that we pass into the get_struct_subset
# function. In this case, occtype==RES1 and num_story <= 2
occtype_list=['RES1-1SNB', 'RES1-2SNB', 'RES1-1SWB', 'RES1-2SWB']
sub_string = 'occtype.isin(@occtype_list) and num_story <= 2'
# We specify the subset step outside of the UNSAFE framework 
# but right now UNSAFE is only able to estimate losses
# for these structures. 
# Down the line, UNSAFE will handle all occupancy types
# and building features that HAZUS does, so the subset
# step will only be called in analyses that are focused
# on certain segments of the building stock. 
nsi_sub = get_struct_subset(nsi_gdf,
                            filter=sub_string,
                            occtype_list=occtype_list)

# For this case study, let us save some memory and just
# write out the single family houses that we just
# subset. You could write out the nsi_gdf GeoDataFrame
# if you'd like to. 
EXP_OUT_FILEP = join(EXP_DIR_I, FIPS, 'nsi_sf.gpkg')
prepare_saving(EXP_OUT_FILEP)
nsi_sub.to_file(EXP_OUT_FILEP, driver='GPKG')

## Reference

We process geospatial reference data (like census tracts and block groups) for three reasons:
1) We will use census tracts to define the unit of aggregation for specifying multinomial distributions for foundation type and number of stories; and
2) Data on social vulnerability, like the Climate and Economic Justice Screening Tool, are available as tabular data so we will link these to the corresponding geospatial reference dataset; and
3) It can be useful to estimate losses at the property-level and then aggregate these estimates to different spatial scales. This is also a common use-case. 

We first define a clip_gdf GeoDataFrame reference to define our study's spatial extent. Here, we use the whole county, but users could upload a specific spatial boundary if they'd like. 

In [ ]:
# We are going to clip reference data to a clip file that
# represents our study boundaries. In this case, it's the county
# of Philadelphia, so we will prepare that as our clip file
county_filep = join(REF_DIR_UZ, NATION, 'county', 'tl_2022_us_county.shp')
county_gdf = gpd.read_file(county_filep)
clip_gdf = county_gdf[county_gdf[REF_ID_NAMES_DICT['county']] == FIPS]

# clip_ref_files will go through all unzipped ref files,
# clip them in the clip file geometry, and write them
# We pass in arguments we defined in the configuration step
# to tell the function where the data can be found (REF_DIR_UZ),
# where is is going (REF_DIR_I), and how to update id names
# (REF_NAMES_DICT). 
clip_ref_files(clip_gdf, FIPS,
               REF_DIR_UZ, REF_DIR_I, REF_NAMES_DICT)

## Physical vulnerability

This is where we call functions to prepare our DDFs for estimating losses under uncertainty. In some analyses, you may only want to use one set of DDFs, so you only have to call one process_DDF() function. In the future, there may be many more functions to draw from, so we decided the workflow would be most transparent if users had to explicitly call a function to process a particular set of DDFs. 

In [ ]:
# For NACCS DDFs, we are just going to call process_naccs
# For HAZUS DDFs, we are going to call process_hazus but also
# specify how to define the uncertainty around these point estimate
# DDFs
# In general, the functions could be expanded to allow the user to
# specify which building types to consider, but right now
# that is baked-in to the implementation in unsafe 
# Both of these functions will write out all the data you need
# for estimating losses later on
# We break it out into two functions because not all analyses
# will want to represent deep uncertainty in DDFs and will
# only call one of the process functions

process_naccs(VULN_DIR_UZ, VULN_DIR_I)

# .3 was used in Zarekarizi et al. 2020
# https://www.nature.com/articles/s41467-020-19188-9
# and we are going to use that for this case study
# We define this here, instead of in the configuration step, 
# because seems helpful to define optional arguments, and arguments
# that can be thought of as uncertain parameters, (like this one)
# closer to the function call. 
UNIF_UNC = .3
process_hazus(VULN_DIR_UZ, VULN_DIR_I, unif_unc=UNIF_UNC)

## Social vulnerability

UNSAFE can ingest CDC Socially Vulnerable Index data and Climate and Economic Justice and Screening Tool data. Users do not have to process data about social vulnerability, but it is a common use case. As such, we developed UNSAFE with preliminary functionality for this type of usage. The function we call below calls subroutines that have code to handle the unique features of the different data. 

In [ ]:
# Process national social vulnerability data
# Tell the function which datasets we want processed
# In this case study, we will use cejst and svi
# which are available nationally and we specified in
# our DOWNLOAD configuration dictionary. 

sovi_list = ['cejst', 'svi']
process_national_sovi(sovi_list, FIPS,
                      VULN_DIR_R, REF_DIR_I, VULN_DIR_I)

## Policy

The National Flood Hazard Layer is required for using HAZUS DDFs because these apply different DDF relationships based on whether a house is in the V zone. In our case study, all houses are in the A or X zone, but this is a core function that is needed for other case studies. 

In [ ]:
# We need NFHL for the ensemble and visualizations
process_nfhl(FIPS,
             POL_DIR_UZ,
             POL_DIR_I)

## Link flood zones and references to structures

We will now link our GeoDataFrame of single family structures to the other spatial data we processed.

In [ ]:
# Link flood zones
# I checked for issues like overlapping flood zones
# resulting in NSI structures in multiple polygons
# and did not find any. That's good, but chances
# are there will be counties where this happens
# and we will need code to handle these consistently for other
# case studies
nfhl_filep = join(POL_DIR_I, FIPS, 'fld_zones.gpkg')
nfhl = gpd.read_file(nfhl_filep)
keep_cols = ['fld_zone', 'fld_ar_id', 'static_bfe']

# This function is designed to work in general
# for any spatial-based data you want to link
# to structures. We demonstrate it with the
# nfhl data. 
get_spatial_var(nsi_sub,
                nfhl,
                'fz',
                FIPS,
                EXP_DIR_I,
                keep_cols)

# Link references
# This will do spatial joins for structures within
# all the reference spatial files (besides county)
# and output a file of fd_id (these are unique strucutre ids)
# linked to all of the reference ids
get_ref_ids(nsi_sub, FIPS,
            REF_ID_NAMES_DICT, REF_DIR_I, EXP_DIR_I)

## Hazard

This is where we link flood depths to structures. We are using the FEMA Flood Risk Database which consists of the 500, 100, 50, and 10 year return periods. We specified in our configuration file that we are only interested in the depth grids corresponding to inland flooding. The purpose of this function is to do point-in-raster sampling for the depth grids indicated by the RET_PERS and HAZ_FILN arguments. Users can add pre-processing code that is unique to the hazard data they are bringing into the UNSAFE framework to call the `get_inundations()` function. For example, if you want to try experimenting with the `full_data_example.ipynb` you can try adding pre-processing code that selects the max inundation across the inland and coastal depth grids in the Flood Risk Database, and then modify the HAZ_FILEN argument to link these depths to the structures. Or, you can modify the RET_PERS argument to serve more generally as a flood scenario indicator and simultaneously modify the HAZ_FILN argument to accommodate reading in the correct files. 

In [ ]:
# Sample the inundation grids and write out the
# fd_id/depths dataframe

# This code looks different than the full data example
# because we created the zip directory in a different way
# than the zip directory you download from the fema
# risk database
haz_dir_uz_clip = join(HAZ_DIR_UZ, 'dg_clipped')
depth_df = get_inundations(nsi_sub,
                           HAZ_CRS, RET_PERS,
                           haz_dir_uz_clip, HAZ_FILEN)

# Because we are processing design-event based flood scenarios,
# we can provide more helpful column names to prepare our loss
# and expected annual loss calculations. We will also
# write out our dataframe. 
ncol = [str(round(100/float(x.replace('_', '.')))) for x in depth_df.columns]
depth_df.columns = ncol

# Write out dataframe that links fd_id to depths
# with columns corresponding to ret_per (i.e. 500, 100, 50, 10)
# in our case study
nsi_depths_out = join(EXP_DIR_I, FIPS, 'nsi_depths.pqt')
prepare_saving(nsi_depths_out)
depth_df.reset_index().to_parquet(nsi_depths_out)

# Generate ensemble

This corresponds to steps 10-12 in the workflow. We include example code for estimating losses without uncertainty on the base dataframe that is used for generating the ensemble. 

In [ ]:
# Get a dataframe conducive for loss estimation
# This procedure is separate from preparing data for the ensemble
# so will just take the county code to load in and merge
# all the relevant data

base_df = get_base_df(FIPS, EXP_DIR_I)

# Generate SOWs based on this dataframe. The function gives
# users the option to specify what to treat as uncertain. It could
# be improved to give the user more customization on the "how" part
# Right now, we require that FFE are represented as uncertain, so
# users don't have the option to toggle this like they do
# with value, stories, and basement. 
# We specify hazus & naccs for the ddfs we want losses estimated
# under
# We specify val, stories, and basement as the features
# we want to represent with uncertainty
# If you generate an ensemble, you are at least considering
# ffe uncertainty from the FFE_DICT
# We estimate losses for the full ensemble. For now, when deep 
# uncertainty is specified in the DDF (i.e. you 
# want to get damages with HAZUS and NACCS) they are estimated on
# the same SOWs and that's returned. No synthesis of 
# deep unceratinties in UNSAFE yet. 
ens_df_f = generate_ensemble(nsi_sub,
                             base_df,
                             ['hazus', 'naccs'],
                             ['ffe', 'val_struct', 'stories', 'basement'],
                             N_SOW,
                             FFE_DICT,
                             COEF_VARIATION,
                             VULN_DIR_I)

Because the flood events we used are defined as return periods, we can estimate expected annual loss. We do subsetting the loss columns and calling the `get_eal()` function on an ordered list from most to least frequent return period (e.g., 10 year event is ordered before the 500 year event). It is easiest to do this by looping through the damage functions. 

In [ ]:
eals = {}
for ddf in ['hazus', 'naccs']:
    col_sub = [x for x in ens_df_f if ddf + '_loss' in x]
    loss_sub = ens_df_f[col_sub]
    # Update column name to remove reference to ddf
    loss_sub.columns = ['_'.join(x.split('_')[1:]) for x in col_sub]
    ret_per_ints = [int(x.split('_')[-1]) for x in col_sub]
    rp_list = sorted(ret_per_ints)
    eals[ddf] = get_eal(loss_sub, rp_list)
eals = pd.DataFrame.from_dict(eals)
eals.columns = [x + '_eal' for x in eals.columns]

In [ ]:
ens_df_out = pd.concat([ens_df_f, eals], axis=1)
ens_out_filep = join(FO, 'ensemble.pqt')
ens_df_out.to_parquet(ens_out_filep)

## Estimate benchmark losses

In [ ]:
# We also want benchmark estimates without uncertainty 
# which we can do with the full_df specified above
nounc_df = benchmark_loss(base_df, VULN_DIR_I)

hazus_def_out_filep = join(FO, 'benchmark_loss.pqt')
prepare_saving(hazus_def_out_filep)
nounc_df.to_parquet(hazus_def_out_filep)

# Checking aggregate results

You can run the code below to compare the histograms of the expected annual loss estimates for both damage functions. You will not get the exact same result each time you run this notebook, but you should get something very similar! You can compare your output to the picture in `examples/phil_frd_partial/fig/example_output.png` 

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
import seaborn as sns

fig, ax = plt.subplots(figsize=(10, 6), dpi=300,
                       nrows=2,
                       sharex=True,
                       gridspec_kw={'hspace': 0},
                       height_ratios=[1,3])
# When we groupby 'sow_ind' we are looking across ensemble members
temp = ens_df_out.groupby(['sow_ind'])[['naccs_eal', 'hazus_eal']].sum()/1e6
temp['naccs_eal'].hist(bins=30, color='blue', alpha=.5, label='NACCS DDFs')
temp['hazus_eal'].hist(bins=30, color='orange', alpha=.5, label='HAZUS DDFs')
ax[1].axvline(nounc_df['eal'].sum()/1e6, color='red', label='No Uncertainty')
ax[0].axvline(nounc_df['eal'].sum()/1e6, color='red', label='No Uncertainty')
ax[1].grid(False)
ax[1].set_xlabel('Total Expected Annual Loss ($ Millions)', size=14)
ax[1].set_ylabel('Number of Ensemble Members', size=14)
ax[1].tick_params(labelsize=12)

temp.columns = ['NACCS', 'HAZUS']
temp_box = temp.melt(value_name='eal',
                     var_name='DDF Type')
sns.boxplot(ax=ax[0],
            data=temp_box,
            x='eal',
            hue='DDF Type',
            legend=False,
            showmeans=True,
            meanprops={'markerfacecolor': 'firebrick',
                        'markeredgecolor': 'black',
                        'marker': 'D'})
ax[0].axis('off')


# Easier to do a custom legend
legend_elements = [Patch(facecolor=sns.color_palette("tab10")[1],
                         label='HAZUS DDFs'),
                    Patch(facecolor=sns.color_palette("tab10")[0],
                         label='NACCS DDFs'),
                   Line2D([0], [0], color='r', lw=2, label='No Uncertainty'),
                   Line2D([0], [0], marker='D', markerfacecolor='firebrick',
                          label='Ensemble Mean',
                          ls='',
                          markeredgecolor='black', markersize=8),]


ax[1].legend(handles=legend_elements,
             loc='upper right',
             fontsize='x-large',
             frameon=False)